# Dealing with 3D objects
Requires Python>=3.7  
Before running, copy or move the cytosim module (cytosim.---.so) to the current folder.
```bash
$ make -j4 pycytosim
$ cp bin/*.so examples/
```

Here we showcase by plotting a network of filaments and motors.  
Requires iPyvolume (conda install -c conda-forge ipyvolume) 

In [1]:
import cytosim
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Just a function to plot the system in 3D
try:
    import ipyvolume as ipv
    def plot_sim(sim):
        f = ipv.figure()
        for fiber in sim.fibers:
            p = fiber.points()
            ipv.plot(p[:,0],p[:,1],p[:,2],color="green")
            ipv.scatter(p[:,0],p[:,1],p[:,2],color="green")
        pos2 = np.array([c.position() for c in sim.couples if c.state()==3])
        pos1 = np.array([c.position() for c in sim.couples if c.state()==1 or c.state()==2])
        if pos2.size>0:
            ipv.scatter(pos2[:,0],pos2[:,1],pos2[:,2],color="red",size=15, marker="sphere")
        if pos1.size>0:
            ipv.scatter(pos1[:,0],pos1[:,1],pos1[:,2],color="magenta",size=15, marker="sphere")
        #ipv.xlim(-10, 10)
        #ipv.ylim(-10, 10)
        #ipv.zlim(-10, 10)
        ipv.squarelim()
        f.camera.position = (0, -20, 0)
        f.camera.fov = 400
        f.camera.lookAt([0,0,0])
        ipv.show()

except:
    print("Could not import ipyvolume")
    def plot_sim(sim):
        print("Ipyvolume not installed")


## Fiber simulation
Here we simulate and a fiber. Each point correspond to the end of a segment.

In [3]:
sim = cytosim.start("../cym/fiber.cym")

In [4]:
# Adding with instructions
sim.add("filament","length = 5",50)
sim.run(10)
plot_sim(sim)

In [5]:
kin_props = "binding_rate=10; binding_range=0.1; \
    unbinding_rate = 0.3 ; unbinding_force = 5 ; \
    activity=move; unloaded_speed = 1.0 ; stall_force = 5 ;hold_growing_end=1 ; "
s = sim.set("hand","kinesin",kin_props)
mot_props = "hand1=kinesin ; hand2=kinesin ; stiffness=100 ; diffusion=100 ; length = 0.05"
sim.set("couple","motor",mot_props)
a = sim.add("motor","",1000)
sim.run(500)
plot_sim(sim)